## EDA
### Users Table
- `id`: Unique identifier for each user (numeric)
- `created_at`: User creation timestamp (ISO date string)
- `attribution_source`: User acquisition source (tiktok, instagram, or organic)
- `country`: User's country (US, TR, or NL)
- `name`: User's name

### User Events Table
- `id`: Unique event identifier (numeric)
- `created_at`: Event timestamp (ISO date string)
- `user_id`: Reference to users table (numeric)
- `event_name`: Type of event (app_install, trial_started, trial_cancelled, subscription_started, subscription_renewed, subscription_cancelled)
- `amount_usd`: Transaction amount in USD (numeric)

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
try:
    conn = sqlite3.connect('papcorns copy.sqlite')
except Exception as e:
    print(e)

In [3]:
users_df = pd.read_sql_query("SELECT * FROM users LIMIT 5;", conn)
print("Users table preview:")
display(users_df)

# Load events table
events_df = pd.read_sql_query("SELECT * FROM user_events LIMIT 5;", conn)
print("\nUser events table preview:")
display(events_df)

Users table preview:


,id,created_at,attribution_source,country,name
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore
2,3,2024-10-15T00:00:00,tiktok,TR,Ivy Anderson
3,4,2024-08-28T00:00:00,tiktok,TR,Alice Brown
4,5,2024-04-03T00:00:00,organic,NL,Bob Moore



User events table preview:


,id,created_at,user_id,event_name,amount_usd
0,1,2024-05-07T00:00:00,1,app_install,None
1,2,2024-05-12T00:00:00,1,trial_started,None
2,3,2024-05-24T00:00:00,1,trial_cancelled,None
3,4,2024-10-12T00:00:00,2,app_install,None
4,5,2024-10-13T00:00:00,2,trial_started,None


In [4]:
users_df = pd.read_sql_query("SELECT * FROM users", conn)

In [5]:
events_df = pd.read_sql_query("SELECT * From user_events ", conn)

In [6]:
users_df.describe

<bound method NDFrame.describe of         id           created_at attribution_source country           name
0        1  2024-05-07T00:00:00          instagram      US      Eve Brown
1        2  2024-10-12T00:00:00          instagram      NL    Frank Moore
2        3  2024-10-15T00:00:00             tiktok      TR   Ivy Anderson
3        4  2024-08-28T00:00:00             tiktok      TR    Alice Brown
4        5  2024-04-03T00:00:00            organic      NL      Bob Moore
...    ...                  ...                ...     ...            ...
997    998  2025-02-01T00:00:00          instagram      TR      Bob Davis
998    999  2024-12-24T00:00:00            organic      NL  Charlie Davis
999   1000  2025-02-13T00:00:00            organic      NL  Jack Anderson
1000  1001  2025-02-16T00:00:00          instagram      US    Bruce Wayne
1001  1002  2025-02-16T00:00:00            organic      TR     Clark Kent

[1002 rows x 5 columns]>

In [7]:
events_df.describe

<bound method NDFrame.describe of         id           created_at  user_id            event_name  amount_usd
0        1  2024-05-07T00:00:00        1           app_install         NaN
1        2  2024-05-12T00:00:00        1         trial_started         NaN
2        3  2024-05-24T00:00:00        1       trial_cancelled         NaN
3        4  2024-10-12T00:00:00        2           app_install         NaN
4        5  2024-10-13T00:00:00        2         trial_started         NaN
...    ...                  ...      ...                   ...         ...
3481  3482  2025-02-25T00:00:00     1000       trial_cancelled         NaN
3482  3483  2025-02-25T00:00:00     1001           app_install         NaN
3483  3484  2025-02-25T00:00:00     1001         trial_started         NaN
3484  3485  2025-02-25T00:00:00     1001  subscription_started        9.99
3485  3486  2025-02-25T00:00:00     1002           app_install         NaN

[3486 rows x 5 columns]>

Let's start with checking missing values

In [8]:
users_df.isna().any()

id                    False
created_at            False
attribution_source    False
country               False
name                  False
dtype: bool

In [9]:
events_df.isna().any()

id            False
created_at    False
user_id       False
event_name    False
amount_usd     True
dtype: bool

Let's explore the missing values of `events_df`

In [10]:
events_df.isna().sum()

id               0
created_at       0
user_id          0
event_name       0
amount_usd    2255
dtype: int64

It turns out, 2255 rows of the 3486 are missing in the `amount_usd` column. This also means that amount of transactions coming from users are 1231.

In [11]:
events_df["event_name"].value_counts()

event_name
app_install               1002
subscription_renewed       750
trial_started              682
subscription_started       481
subscription_cancelled     370
trial_cancelled            201
Name: count, dtype: int64

Since `subscription_renewed` (750) + `subscription_cancelled` (481) = 1231, this would mean the users who subscribed to the service all paid for the service, and other users who didn't are the ones that show up as `Nan`. Let's check by checking if there are any users who paid for the service but amount_used is na.

In [12]:
subscription_events = events_df[events_df["event_name"].isin(
    # selecting events that should be paid
    ['subscription_started', 'subscription_renewed'])]
# selecting events that dont have any transactions

missing_amount = subscription_events[subscription_events["amount_usd"].isna()]

print(f"Total subscription events: {len(subscription_events)}")

print(f"Subscription events with missing amount: {len(missing_amount)}")

Total subscription events: 1231
Subscription events with missing amount: 0


In [13]:
events_df["amount_usd"].nunique()

3

In [14]:
unique_values = events_df["amount_usd"].unique()
print(unique_values)

[ nan 8.99 4.99 9.99]


In [15]:
users_df["id"].nunique()

1002

In [16]:
events_df["user_id"].nunique()

1002

It seems we have 1002 users in total.

We can understand that there are 3 subscription tiers.I'll join the two tables for easier anaylsis

In [17]:
combined_df = pd.merge(
    events_df,
    users_df,
    left_on='user_id',
    right_on='id',
    how='left',
    suffixes=('_event', '_user')
)

# Renaming columns to be more clear after the merge
combined_df = combined_df.rename(columns={
    'id_event': 'event_id', 
    'created_at_event': 'event_timestamp',
    'id_user': 'user_id',
    'created_at_user': 'user_created_at'
})

# Display the first few rows of the combined dataframe
print(combined_df.head())

   event_id      event_timestamp  user_id       event_name  amount_usd  \
0         1  2024-05-07T00:00:00        1      app_install         NaN   
1         2  2024-05-12T00:00:00        1    trial_started         NaN   
2         3  2024-05-24T00:00:00        1  trial_cancelled         NaN   
3         4  2024-10-12T00:00:00        2      app_install         NaN   
4         5  2024-10-13T00:00:00        2    trial_started         NaN   

   user_id      user_created_at attribution_source country         name  
0        1  2024-05-07T00:00:00          instagram      US    Eve Brown  
1        1  2024-05-07T00:00:00          instagram      US    Eve Brown  
2        1  2024-05-07T00:00:00          instagram      US    Eve Brown  
3        2  2024-10-12T00:00:00          instagram      NL  Frank Moore  
4        2  2024-10-12T00:00:00          instagram      NL  Frank Moore  


As a result, we have duplicate columns. Let's address this issue by dropping one of the `user_id`

In [18]:
combined_df= combined_df.loc[:,~combined_df.columns.duplicated()].copy() # see https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns 

In [19]:
print(combined_df.head())

   event_id      event_timestamp  user_id       event_name  amount_usd  \
0         1  2024-05-07T00:00:00        1      app_install         NaN   
1         2  2024-05-12T00:00:00        1    trial_started         NaN   
2         3  2024-05-24T00:00:00        1  trial_cancelled         NaN   
3         4  2024-10-12T00:00:00        2      app_install         NaN   
4         5  2024-10-13T00:00:00        2    trial_started         NaN   

       user_created_at attribution_source country         name  
0  2024-05-07T00:00:00          instagram      US    Eve Brown  
1  2024-05-07T00:00:00          instagram      US    Eve Brown  
2  2024-05-07T00:00:00          instagram      US    Eve Brown  
3  2024-10-12T00:00:00          instagram      NL  Frank Moore  
4  2024-10-12T00:00:00          instagram      NL  Frank Moore  


I was gonna used `combined_df` for the tasks, but I realized if I did that there'd be little to none SQL in this case which is not what was requested.

## Tasks

### 1. Calculate the total revenue generated from subscriptions for each country.

For revenue coming from each country, we're gonna start by selecting the country field from `users` and sum the `amound_usd`. We're gonna be needing paid users. That means we're gonna select users who have their `event_name` field either as `subscription_started` or `subscription_renewed`. Also a join operation is needed since we're using two tables 

In [20]:
query = """
SELECT 
    country,
    SUM(CASE 
        WHEN event_name IN ("subscription_started", "subscription_renewed") 
        THEN amount_usd 
    END) as total_revenue
FROM users u
LEFT JOIN user_events e ON u.id = user_id
GROUP BY u.country
ORDER BY total_revenue
"""
revenue_per_country = pd.read_sql_query(query, conn)
revenue_per_country

,country,total_revenue
0,TR,2275.44
1,NL,3335.29
2,US,4035.96


### 2. Calculate the total number of trials given to users who came from Instagram.

We're gonna need aliases for the tables as column they both have a column named id and it'd be problematic when doing a join operation in the query. We're gonna be needing all entries that fulfill two conditions. Users whose `attribution source` is instagram, and users who started a trial.

In [21]:
query = """
SELECT COUNT(*) as total_trials
FROM users u
JOIN user_events e ON u.id = e.user_id
WHERE u.attribution_source = "instagram"
AND e.event_name = "trial_started"
"""
ig_trials = pd.read_sql_query(query, conn)
ig_trials

,total_trials
0,210


Let's be sure about attribution source labelling is correct and there aren't any typos or any other issues

In [22]:
print(users_df["attribution_source"].unique())

['instagram' 'tiktok' 'organic']


This query could be further enhanced by not including the users who didn't subscribe to the service and ended the trial 

### 3. Create a new column named `acquisition_channel` by categorizing users based on their `attribution_source`:

* 'Paid': users from instagram or tiktok

* "Organic": users from organic sources

First lets start with exploring the possible sources of users

In [23]:
combined_df["attribution_source"].unique()

array(['instagram', 'tiktok', 'organic'], dtype=object)

Double checking if there are any missing values

In [24]:
combined_df["attribution_source"].isna().any()

np.False_

In [25]:
combined_df["attribution_source"]

0       instagram
1       instagram
2       instagram
3       instagram
4       instagram
          ...    
3481      organic
3482    instagram
3483    instagram
3484    instagram
3485      organic
Name: attribution_source, Length: 3486, dtype: object

There are no missing values, and we have 3 possible sources. `instagram`, 
`tiktok`,`organic`



There are multiple ways to determine if we should categorize an user as a "paid" user. First we can check if `event_name IN ("subscription_started", "subscription_renewed") ` like we did earlier, and second we can check `amount_usd`. Let's do both

This query finds users who either started or renewed the subscription

In [ ]:
query = """
SELECT 
    u.id,
    u.attribution_source,
    u.country,
    u.name,
    'Paid' AS acquisition_channel
FROM 
    users u
INNER JOIN 
    user_events e ON u.id = e.user_id
WHERE
    e.event_name IN ('subscription_started', 'subscription_renewed') 
GROUP BY
    u.id, u.attribution_source, u.country, u.name
HAVING
    COUNT(DISTINCT e.event_name) = 2
"""


acq_ch = pd.read_sql_query(query, conn)
print(acq_ch)

      id attribution_source country            name acquisition_channel
0      2          instagram      NL     Frank Moore                Paid
1      3             tiktok      TR    Ivy Anderson                Paid
2      8             tiktok      TR     Alice Moore                Paid
3      9            organic      US  Frank Anderson                Paid
4     10            organic      US  David Anderson                Paid
..   ...                ...     ...             ...                 ...
365  991            organic      TR     Jack Miller                Paid
366  992          instagram      US     Grace Jones                Paid
367  995          instagram      US     Frank Smith                Paid
368  996            organic      TR   Jack Anderson                Paid
369  999            organic      NL   Charlie Davis                Paid

[370 rows x 5 columns]


This query finds ALL users who ever paid any amount (481 users)

In [ ]:
query = """
SELECT 
    u.id,
    u.attribution_source,
    u.country,
    u.name,
    "Paid" AS acquisition_channel
FROM 
    users u
INNER JOIN 
    user_events e ON u.id = e.user_id
WHERE
    e.amount_usd > 0
GROUP BY
    u.id, u.attribution_source, u.country, u.name
"""

acq_ch = pd.read_sql_query(query, conn)
print(acq_ch)

       id attribution_source country            name acquisition_channel
0       2          instagram      NL     Frank Moore                Paid
1       3             tiktok      TR    Ivy Anderson                Paid
2       8             tiktok      TR     Alice Moore                Paid
3       9            organic      US  Frank Anderson                Paid
4      10            organic      US  David Anderson                Paid
..    ...                ...     ...             ...                 ...
476   992          instagram      US     Grace Jones                Paid
477   995          instagram      US     Frank Smith                Paid
478   996            organic      TR   Jack Anderson                Paid
479   999            organic      NL   Charlie Davis                Paid
480  1001          instagram      US     Bruce Wayne                Paid

[481 rows x 5 columns]


We see that there are 370 users who started a subscription and renewed, and 481 who made at least one payment. It'd be more accurate to use the first query as they are users who are currently paying

In [ ]:
query = """
SELECT
    u.id,
    u.attribution_source,
    u.country,
    u.name,
    CASE
        WHEN u.attribution_source IN ('instagram', 'tiktok') THEN 'Paid'
        WHEN u.attribution_source = 'organic' THEN 'Organic'
    END AS acquisition_channel
FROM
    users u
INNER JOIN
    user_events e ON u.id = e.user_id
WHERE
    e.event_name IN ("subscription_started", "subscription_renewed")
GROUP BY
    u.id, u.attribution_source, u.country, u.name
HAVING
    COUNT(DISTINCT e.event_name) = 2
"""

acq_ch = pd.read_sql_query(query, conn)
print(acq_ch)

      id attribution_source country            name acquisition_channel
0      2          instagram      NL     Frank Moore                Paid
1      3             tiktok      TR    Ivy Anderson                Paid
2      8             tiktok      TR     Alice Moore                Paid
3      9            organic      US  Frank Anderson             Organic
4     10            organic      US  David Anderson             Organic
..   ...                ...     ...             ...                 ...
365  991            organic      TR     Jack Miller             Organic
366  992          instagram      US     Grace Jones                Paid
367  995          instagram      US     Frank Smith                Paid
368  996            organic      TR   Jack Anderson             Organic
369  999            organic      NL   Charlie Davis             Organic

[370 rows x 5 columns]


### 4. Analyze the trial-to-subscription conversion rate
* Calculate the overall conversion rate
* Break down the conversion rate by attribution_source

Let's start by getting the users who started a trial 

In [29]:
trial_users_query = """
SELECT 
    u.attribution_source,
    COUNT(DISTINCT u.id) AS trial_users
FROM 
    users u
INNER JOIN 
    user_events e ON u.id = e.user_id
WHERE 
    e.event_name = "trial_started"
GROUP BY 
    u.attribution_source
"""

Then, let's find the "paid" users using the same query from pervious task

In [30]:
paid_users_query = """
SELECT 
    u.attribution_source,
    COUNT(DISTINCT u.id) AS paid_users
FROM 
    users u
INNER JOIN 
    user_events e ON u.id = e.user_id
WHERE
    e.event_name IN ("subscription_started", "subscription_renewed") 
GROUP BY
    u.attribution_source
HAVING
    COUNT(DISTINCT e.event_name) = 2
"""

This query would return 370 paid users 

Let's merge them to a dataframe for finding the rates using previous queries. I chose to convert them into dataframes since for this situation, the resulting dataframe would fit any machine and isnt't big enough to cause any issues. For a production environment it could be more optimal to not do that.

In [31]:
trial_users_df = pd.read_sql_query(trial_users_query, conn)
paid_users_df = pd.read_sql_query(paid_users_query, conn)

conversion_rates = pd.merge(
    trial_users_df, 
    paid_users_df, 
    on="attribution_source", 
    how="left"
)

In [32]:
total_trials = conversion_rates["trial_users"].sum()
total_conversions = conversion_rates["paid_users"].sum()
overall_conversion_rate = (total_conversions / total_trials * 100).round(2)

In [33]:
print("Trial-to-Subscription Conversion Rate Analysis")
print("=" * 50)
print(f"Overall Conversion Rate: {overall_conversion_rate}%")
print("\nConversion Rate by Attribution Source:")
print(conversion_rates)

Trial-to-Subscription Conversion Rate Analysis
Overall Conversion Rate: 70.53%

Conversion Rate by Attribution Source:
  attribution_source  trial_users  paid_users
0          instagram          210         149
1            organic          236         169
2             tiktok          236         163


Out of 682 users who started a trial, 70.53% of them became paid users resulting in a 70.53% overall conversion rate

### 5. Calculate the median subscription duration (in months) for each country

In [ ]:
subscription_duration_query = """
SELECT
    u.country,
    u.id AS user_id,
    MIN(CASE WHEN e.event_name = "subscription_started" THEN e.created_at END) AS start_date,
    MAX(CASE WHEN e.event_name = "subscription_cancelled" THEN e.created_at ELSE CURRENT_DATE END) AS end_date
FROM
    users u
INNER JOIN
    user_events e ON u.id = e.user_id
WHERE
    e.event_name IN ("subscription_started", "subscription_cancelled")
GROUP BY
    u.country, u.id
HAVING
    start_date IS NOT NULL
"""

subscription_data = pd.read_sql_query(subscription_duration_query, conn)


subscription_data["start_date"] = pd.to_datetime(subscription_data["start_date"], format="ISO8601")
subscription_data["end_date"] = pd.to_datetime(subscription_data["end_date"], format="ISO8601")


subscription_data["duration_days"] = (subscription_data["end_date"] - subscription_data["start_date"]).dt.days


median_duration_by_country = subscription_data.groupby("country")["duration_days"].median().reset_index()
median_duration_by_country = median_duration_by_country.sort_values(by="duration_days", ascending=False)

print("Median Subscription Duration by Country (in days):")
print(median_duration_by_country)

Median Subscription Duration by Country (in days):
  country  duration_days
0      NL          221.0
2      US          213.0
1      TR          198.0


### 6. Calculate the Average Lifetime Value (LTV) by country

In [ ]:
ltv_query = """
SELECT 
    u.country,
    u.id AS user_id,
    SUM(e.amount_usd) AS total_revenue
FROM 
    users u
JOIN 
    user_events e ON u.id = e.user_id
WHERE 
    e.amount_usd > 0
GROUP BY 
    u.country, u.id
"""

user_revenue_df = pd.read_sql_query(ltv_query, conn)


ltv_by_country = user_revenue_df.groupby("country")["total_revenue"].mean().reset_index()
ltv_by_country.rename(columns={"total_revenue": "average_ltv"}, inplace=True)
ltv_by_country["average_ltv"] = ltv_by_country["average_ltv"].round(2)

In [44]:
overall_ltv = user_revenue_df["total_revenue"].mean()
ltv_by_country = ltv_by_country.sort_values("average_ltv", ascending=False)

In [45]:
print("Average Lifetime Value (LTV) Analysis")
print("=" * 50)
print(f"Overall Average LTV: ${overall_ltv}")
print("\nAverage LTV by Country:")
print(ltv_by_country)

Average Lifetime Value (LTV) Analysis
Overall Average LTV: $20.055488565488567

Average LTV by Country:
  country  average_ltv
2      US        25.07
0      NL        22.38
1      TR        13.31
